# 06 Semantic Kernel | 06 Using Logs

## Intro - Using LoggerFactory

The logging mechanism in this sample makes use of the [ILogger](https://learn.microsoft.com/en-us/dotnet/api/microsoft.extensions.logging.ilogger?view=dotnet-plat-ext-7.0) interface from the [Microsoft.Extensions.Logging](https://learn.microsoft.com/en-us/dotnet/api/microsoft.extensions.logging?view=dotnet-plat-ext-7.0) namespace. The [KernelBuilder.WithLoggerFactory(ILoggerFactory)](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.kernelbuilder.withloggerfactory?view=semantic-kernel-dotnet#microsoft-semantickernel-kernelbuilder-withloggerfactory(microsoft-extensions-logging-iloggerfactory)) method expects an [ILoggerFactory](https://learn.microsoft.com/en-us/dotnet/api/microsoft.extensions.logging.iloggerfactory?view=dotnet-plat-ext-7.0) parameter that allows the Semantic Kernel to generate a logger to facilitate a more accurate logging and streamlined control over log filtering across various classes.

## Step 1: Import nuget packages

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: Microsoft.Extensions.Logging, 6.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 6.0.0"
#r "nuget: Microsoft.Extensions.Logging.Debug, 6.0.0"

using Azure; 
using DotNetEnv;
using Microsoft.SemanticKernel;

#### Expected output

```
Installed Packages
DotNetEnv, 2.5.0
Microsoft.Extensions.Logging, 6.0.0
Microsoft.Extensions.Logging.Console, 6.0.0
Microsoft.Extensions.Logging.Debug, 6.0.0
Microsoft.SemanticKernel, 1.0.0-beta1
```

## Step 2 (optional): Create a logger and a logger provider

The main use here is to showcase the ability to use logger with the semantic kernel. The shallow implementation provided is to enable writing to the console within a Polyglot notebook.

In [ ]:
using Microsoft.Extensions.Logging;

public class CustomConsoleLogger : ILogger
{

    private readonly string _name;
    public CustomConsoleLogger(string name)
    {
        _name = name;
    }

    public IDisposable? BeginScope<TState>(TState state) where TState : notnull
    {
        return null; 
    }

    public bool IsEnabled(LogLevel logLevel)
    {
        return true; 
    }

    public void Log<TState>(LogLevel logLevel, EventId eventId, TState state, Exception? exception, Func<TState, Exception?, string> formatter)
    {
        if (!IsEnabled(logLevel)) return; 

        var logMessage = formatter(state, exception);
        Console.WriteLine($"{logLevel}: {_name} - {logMessage}");
    }
}

public class ConsoleLoggerProvider : ILoggerProvider
{
    public void Dispose()
    {
        // Dispose resources if any
    }

    public ILogger CreateLogger(string categoryName)
    {
        return new CustomConsoleLogger(categoryName);
    }
}

Console.WriteLine($"Custom console logger created...");

Extected output: 

```
Custom console logger created...
```

## Step 3: Create a Logger

In [ ]:
var loggerFactory = LoggerFactory
    .Create(builder => {
        builder.SetMinimumLevel(LogLevel.Trace)
            .AddFilter("Microsoft", LogLevel.Trace)
            .AddFilter("OpenAI.StarterKit", LogLevel.Trace)
            .AddProvider(new ConsoleLoggerProvider()); 
        }
    );

Console.WriteLine($"Custom Logger 'ConsoleLoggerProvider' created...");

Expected output:
```
Custom Logger 'ConsoleLoggerProvider' created...
```

## Step 4: Creating the Semantic Kernel

An instance of Semantic Kernel can be created using the KernelBuilder object provided by the Semantic Kernel SDK. It acts as the centralized point for all .NET functionality that want to interact with Semantic Kernel functionality or concepts.

It abstracts e.g., models from the OpenAI GPT family and can communicate with Azure OpenAI LLM instances as well as LLMs deployed on OpenAI. In the sample we use models deployed on Azure.

In [ ]:
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";

IKernel kernel = new KernelBuilder()
    .WithLoggerFactory(loggerFactory)
    .WithAzureChatCompletionService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: chatCompletionDeploymentName)
    .Build();

Console.WriteLine($"Semantic Kernel initialized...");

Expected output: 

```
Semantic Kernel initialized...
```

## Step 4: Get logger instance from Semantic Kernel

In [ ]:
var logger = kernel.LoggerFactory.CreateLogger("OpenAI.StarterKit");

logger.LogInformation("Logger instance retreived from Semantic Kernel...");

Expected output: 

```
Information: OpenAI.StarterKit - Logger instance retreived from Semantic Kernel...
```

## Step 5: Import functions

In [ ]:
using System.IO; 

string pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "PluginLibrary");

string plugInName = "CreateSummary";
string functionName = "SimpleSummary";
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, plugInName);

logger.LogInformation($"Semantic functions imported from directory '{pluginsDirectory}'...");

Expected output:

Note, few of the log messages were created by a semantic kernel.

```
Installed Packages
DotNetEnv, 2.5.0
Microsoft.Extensions.Logging, 6.0.0
Microsoft.Extensions.Logging.Console, 6.0.0
Microsoft.Extensions.Logging.Debug, 6.0.0
Microsoft.SemanticKernel, 1.0.0-beta1

Semantic Kernel initialized...
Trace: Microsoft.SemanticKernel.IKernel - Config SimpleSummary: {
  "schema": 1,
  "description": "Creates a short summary of provided text",
  "input": {
    "parameters": []
  },
  "models": [
    {
      "service_id": null,
      "model_id": null,
      "max_tokens": 100,
      "temperature": 0.1,
      "top_p": 0.0,
      "presence_penalty": 0.0,
      "frequency_penalty": 0.0
    }
  ],
  "type": "completion",
  "completion": {
    "service_id": null,
    "model_id": null,
    "max_tokens": 100,
    "temperature": 0.1,
    "top_p": 0.0,
    "presence_penalty": 0.0,
  "default_services": []
}
Trace: Microsoft.SemanticKernel.IKernel - Registering function CreateSummary.SimpleSummary loaded from c:\Sourcen\GitHubProjects\OpenAI.StarterKit\docs\07_SemanticKernel\PluginLibrary\CreateSummary\SimpleSummary
Information: OpenAI.StarterKit - Semantic functions imported from directory 'XXXX\docs\07_SemanticKernel\PluginLibrary'...
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

Custom console logger created...
Custom Logger 'ConsoleLoggerProvider' created...
Information: OpenAI.StarterKit - Logger instance retreived from Semantic Kernel......
```

## Step 6: Execute functions

In [ ]:
using System.IO;

//Read text which should be summarized from file
string assetsFolder = @"../../assets";
string input = await File.ReadAllTextAsync(Path.Combine(assetsFolder,"docs","06_SemanticKernel", "aci_documentation.txt"));

ISKFunction sKFunction = kernel.Functions.GetFunction(plugInName, functionName);
var kernelResult = await kernel.RunAsync(input, sKFunction);

Console.WriteLine($"Summary: {kernelResult.GetValue<string>().Substring(0,50)}...");

#### Expected output

Note, few of the log messages were created by a semantic kernel core modules.

```
Trace: Microsoft.SemanticKernel.TemplateEngine.Basic.BasicPromptTemplateEngine - Rendering string template: You are an AI assistant that creates short and accurate summaries of provided text.
The summary must not be more than 2 short sentences. Stop responding after you've created the two sentences. Reply in full sentences and avoid bullet points.

---
User: {{$input}}
Assistant: 
---
Trace: Microsoft.SemanticKernel.TemplateEngine.Basic.BasicPromptTemplateEngine - Extracting blocks from template: You are an AI assistant that creates short and accurate summaries of provided text.
The summary must not be more than 2 short sentences. Stop responding after you've created the two sentences. Reply in full sentences and avoid bullet points.

---
User: {{$input}}
Assistant: 
---
Trace: Microsoft.SemanticKernel.TemplateEngine.Basic.BasicPromptTemplateEngine - Rendering list of 3 blocks
Trace: Microsoft.SemanticKernel.TemplateEngine.Basic.BasicPromptTemplateEngine - Rendered prompt: You are an AI assistant that creates short and accurate summaries of provided text.
The summary must not be more than 2 short sentences. Stop responding after you've created the two sentences. Reply in full sentences and avoid bullet points.

---
User: Configure a GitHub Action to create a container instance Article • 05/03/2023 GitHub Actions is a suite of features in GitHub to automate your software development workflows in the same place you store code and collaborate on pull requests and issues. Use the Deploy to Azure Container Instances GitHub Actions to automate deployment of a single container to Azure Container Instances. The action allows you to set properties for a container instance similar to those in the az container create command. This article shows how to set up a workflow in a GitHub repo that performs the following actions: Build an image from a Dockerfile Push the image to an Azure container registry Deploy the container image to an Azure container instance This article shows two ways to set up the workflow: Configure GitHub workflow - Create a workflow in a GitHub repo using the Deploy to Azure Container Instances action and other actions. Use CLI extension - Use the az container app up command in the Deploy to Azure extension in the Azure CLI. This command streamlines creation of the GitHub workflow and deployment steps. ） Important The GitHub Actions for Azure Container Instances is currently in preview. Previews are made available to you on the condition that you agree to the supplemental terms of use . Some aspects of this feature may change prior to general availability (GA). Prerequisites GitHub account - Create an account on https://github.com if you don't already have one. Azure CLI - You can use the Azure Cloud Shell or a local installation of the Azure CLI to complete the Azure CLI steps. If you need to install or upgrade, see Install
Azure CLI. Azure container registry - If you don't have one, create an Azure container registry in the Basic tier using the Azure CLI, Azure portal, or other methods. Take note of the resource group used for the deployment, which is used for the GitHub workflow. Set up repo For the examples in this article, use GitHub to fork the following repository: https://github.com/Azure-Samples/acr-build-helloworld-node This repo contains a Dockerfile and source files to create a container image of a small web app. Ensure Actions is enabled for your repository. Navigate to your forked repository and select Settings > Actions. In Actions permissions, ensure that Allow all actions is selected. Configure GitHub workflow Create credentials for Azure authentication Service principal In the GitHub workflow, you need to supply Azure credentials to authenticate to the Azure CLI. The following example creates a service principal with the Contributor role scoped to the resource group for your container registry. First, get the resource ID of your resource group. Substitute the name of your group in the following az group show command: Azure CLI groupId=$(az group show \ --name <resource-group-name> \ --query id --output tsv) Use az ad sp create-for-rbac to create the service principal:
Assistant: 
---
Information: Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion.AzureChatCompletion - Action: GetCompletionsAsync. Azure OpenAI Deployment Name: yd-gpt35-turbo.
Information: Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion.AzureChatCompletion - Prompt tokens: 631. Completion tokens: 45. Total tokens: 676.
Summary: This article explains how to use the Deploy to Azu...
```